In [1]:
import torch
import torch.nn as nn
import torchvisiontorchvision.transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 784  #####   For Mnist


In [4]:
# Download data, if not already at  '\root' location

train_dataset = torchvision.datasets.MNIST(root='../datasets/', train = True,
                                          transform=torchvision.transforms.ToTensor(),
                                          download = True)
test_dataset = torchvision.datasets.MNIST(root='../datasets/', train = False,
                                          transform=torchvision.transforms.ToTensor(),
                                          download = True)
#  Create iterators

train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=32,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=32,
                                          shuffle = False)


0it [00:00, ?it/s]

9920512it [00:05, 1968827.58it/s]                             


Extracting ../datasets/MNIST/raw/train-images-idx3-ubyte.gz to ../datasets/MNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 111122.75it/s]           
0it [00:00, ?it/s]

Extracting ../datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ../datasets/MNIST/raw


1654784it [00:00, 1797969.88it/s]                             
0it [00:00, ?it/s]

Extracting ../datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ../datasets/MNIST/raw


8192it [00:00, 18298.79it/s]            


Extracting ../datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../datasets/MNIST/raw
Processing...
Done!


In [10]:
num_classes = 10
n_epochs = 25
steps = len(train_loader)
hidden = 500    #######    for 1 hidden layer
class Net(nn.Module):
    
    def __init__(self, input_size, hidden, num_classes):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_size, hidden)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden,num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = Net(input_size, hidden, num_classes).to(device)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

for e in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images  = images.reshape(-1,784).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss_val = loss(outputs, labels)
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()
        
        if (e+1)%5==0:
            print('Epoch [{}/{}], step [{}/{}], loss-----> {:.3f}'.format(
                e+1,n_epochs, i, steps, loss_val.item()))
        
        

Epoch [5/25], step [0/1875], loss-----> 0.358
Epoch [5/25], step [1/1875], loss-----> 0.464
Epoch [5/25], step [2/1875], loss-----> 0.197
Epoch [5/25], step [3/1875], loss-----> 0.192
Epoch [5/25], step [4/1875], loss-----> 0.266
Epoch [5/25], step [5/1875], loss-----> 0.248
Epoch [5/25], step [6/1875], loss-----> 0.215
Epoch [5/25], step [7/1875], loss-----> 0.490
Epoch [5/25], step [8/1875], loss-----> 0.324
Epoch [5/25], step [9/1875], loss-----> 0.116
Epoch [5/25], step [10/1875], loss-----> 0.482
Epoch [5/25], step [11/1875], loss-----> 0.276
Epoch [5/25], step [12/1875], loss-----> 0.694
Epoch [5/25], step [13/1875], loss-----> 0.106
Epoch [5/25], step [14/1875], loss-----> 0.191
Epoch [5/25], step [15/1875], loss-----> 0.376
Epoch [5/25], step [16/1875], loss-----> 0.191
Epoch [5/25], step [17/1875], loss-----> 0.423
Epoch [5/25], step [18/1875], loss-----> 0.409
Epoch [5/25], step [19/1875], loss-----> 0.183
Epoch [5/25], step [20/1875], loss-----> 0.109
Epoch [5/25], step [21/

Epoch [5/25], step [188/1875], loss-----> 0.289
Epoch [5/25], step [189/1875], loss-----> 0.085
Epoch [5/25], step [190/1875], loss-----> 0.230
Epoch [5/25], step [191/1875], loss-----> 0.214
Epoch [5/25], step [192/1875], loss-----> 0.334
Epoch [5/25], step [193/1875], loss-----> 0.250
Epoch [5/25], step [194/1875], loss-----> 0.412
Epoch [5/25], step [195/1875], loss-----> 0.179
Epoch [5/25], step [196/1875], loss-----> 0.282
Epoch [5/25], step [197/1875], loss-----> 0.140
Epoch [5/25], step [198/1875], loss-----> 0.318
Epoch [5/25], step [199/1875], loss-----> 0.491
Epoch [5/25], step [200/1875], loss-----> 0.174
Epoch [5/25], step [201/1875], loss-----> 0.260
Epoch [5/25], step [202/1875], loss-----> 0.425
Epoch [5/25], step [203/1875], loss-----> 0.406
Epoch [5/25], step [204/1875], loss-----> 0.085
Epoch [5/25], step [205/1875], loss-----> 0.133
Epoch [5/25], step [206/1875], loss-----> 0.250
Epoch [5/25], step [207/1875], loss-----> 0.227
Epoch [5/25], step [208/1875], loss-----

Epoch [5/25], step [365/1875], loss-----> 0.235
Epoch [5/25], step [366/1875], loss-----> 0.266
Epoch [5/25], step [367/1875], loss-----> 0.232
Epoch [5/25], step [368/1875], loss-----> 0.176
Epoch [5/25], step [369/1875], loss-----> 0.403
Epoch [5/25], step [370/1875], loss-----> 0.432
Epoch [5/25], step [371/1875], loss-----> 0.289
Epoch [5/25], step [372/1875], loss-----> 0.318
Epoch [5/25], step [373/1875], loss-----> 0.307
Epoch [5/25], step [374/1875], loss-----> 0.287
Epoch [5/25], step [375/1875], loss-----> 0.183
Epoch [5/25], step [376/1875], loss-----> 0.457
Epoch [5/25], step [377/1875], loss-----> 0.190
Epoch [5/25], step [378/1875], loss-----> 0.132
Epoch [5/25], step [379/1875], loss-----> 0.306
Epoch [5/25], step [380/1875], loss-----> 0.162
Epoch [5/25], step [381/1875], loss-----> 0.413
Epoch [5/25], step [382/1875], loss-----> 0.495
Epoch [5/25], step [383/1875], loss-----> 0.224
Epoch [5/25], step [384/1875], loss-----> 0.164
Epoch [5/25], step [385/1875], loss-----

Epoch [5/25], step [540/1875], loss-----> 0.237
Epoch [5/25], step [541/1875], loss-----> 0.394
Epoch [5/25], step [542/1875], loss-----> 0.248
Epoch [5/25], step [543/1875], loss-----> 0.203
Epoch [5/25], step [544/1875], loss-----> 0.229
Epoch [5/25], step [545/1875], loss-----> 0.518
Epoch [5/25], step [546/1875], loss-----> 0.175
Epoch [5/25], step [547/1875], loss-----> 0.236
Epoch [5/25], step [548/1875], loss-----> 0.191
Epoch [5/25], step [549/1875], loss-----> 0.381
Epoch [5/25], step [550/1875], loss-----> 0.170
Epoch [5/25], step [551/1875], loss-----> 0.197
Epoch [5/25], step [552/1875], loss-----> 0.313
Epoch [5/25], step [553/1875], loss-----> 0.196
Epoch [5/25], step [554/1875], loss-----> 0.118
Epoch [5/25], step [555/1875], loss-----> 0.392
Epoch [5/25], step [556/1875], loss-----> 0.098
Epoch [5/25], step [557/1875], loss-----> 0.237
Epoch [5/25], step [558/1875], loss-----> 0.184
Epoch [5/25], step [559/1875], loss-----> 0.465
Epoch [5/25], step [560/1875], loss-----

Epoch [5/25], step [721/1875], loss-----> 0.198
Epoch [5/25], step [722/1875], loss-----> 0.154
Epoch [5/25], step [723/1875], loss-----> 0.176
Epoch [5/25], step [724/1875], loss-----> 0.273
Epoch [5/25], step [725/1875], loss-----> 0.138
Epoch [5/25], step [726/1875], loss-----> 0.443
Epoch [5/25], step [727/1875], loss-----> 0.156
Epoch [5/25], step [728/1875], loss-----> 0.379
Epoch [5/25], step [729/1875], loss-----> 0.204
Epoch [5/25], step [730/1875], loss-----> 0.171
Epoch [5/25], step [731/1875], loss-----> 0.362
Epoch [5/25], step [732/1875], loss-----> 0.065
Epoch [5/25], step [733/1875], loss-----> 0.284
Epoch [5/25], step [734/1875], loss-----> 0.144
Epoch [5/25], step [735/1875], loss-----> 0.286
Epoch [5/25], step [736/1875], loss-----> 0.598
Epoch [5/25], step [737/1875], loss-----> 0.301
Epoch [5/25], step [738/1875], loss-----> 0.179
Epoch [5/25], step [739/1875], loss-----> 0.121
Epoch [5/25], step [740/1875], loss-----> 0.302
Epoch [5/25], step [741/1875], loss-----

...

Epoch [25/25], step [1186/1875], loss-----> 0.045
Epoch [25/25], step [1187/1875], loss-----> 0.039
Epoch [25/25], step [1188/1875], loss-----> 0.022
Epoch [25/25], step [1189/1875], loss-----> 0.180
Epoch [25/25], step [1190/1875], loss-----> 0.045
Epoch [25/25], step [1191/1875], loss-----> 0.017
Epoch [25/25], step [1192/1875], loss-----> 0.188
Epoch [25/25], step [1193/1875], loss-----> 0.036
Epoch [25/25], step [1194/1875], loss-----> 0.042
Epoch [25/25], step [1195/1875], loss-----> 0.150
Epoch [25/25], step [1196/1875], loss-----> 0.024
Epoch [25/25], step [1197/1875], loss-----> 0.042
Epoch [25/25], step [1198/1875], loss-----> 0.050
Epoch [25/25], step [1199/1875], loss-----> 0.142
Epoch [25/25], step [1200/1875], loss-----> 0.032
Epoch [25/25], step [1201/1875], loss-----> 0.039
Epoch [25/25], step [1202/1875], loss-----> 0.210
Epoch [25/25], step [1203/1875], loss-----> 0.213
Epoch [25/25], step [1204/1875], loss-----> 0.018
Epoch [25/25], step [1205/1875], loss-----> 0.042


Epoch [25/25], step [1354/1875], loss-----> 0.036
Epoch [25/25], step [1355/1875], loss-----> 0.070
Epoch [25/25], step [1356/1875], loss-----> 0.362
Epoch [25/25], step [1357/1875], loss-----> 0.041
Epoch [25/25], step [1358/1875], loss-----> 0.180
Epoch [25/25], step [1359/1875], loss-----> 0.269
Epoch [25/25], step [1360/1875], loss-----> 0.059
Epoch [25/25], step [1361/1875], loss-----> 0.022
Epoch [25/25], step [1362/1875], loss-----> 0.080
Epoch [25/25], step [1363/1875], loss-----> 0.073
Epoch [25/25], step [1364/1875], loss-----> 0.110
Epoch [25/25], step [1365/1875], loss-----> 0.032
Epoch [25/25], step [1366/1875], loss-----> 0.077
Epoch [25/25], step [1367/1875], loss-----> 0.070
Epoch [25/25], step [1368/1875], loss-----> 0.021
Epoch [25/25], step [1369/1875], loss-----> 0.018
Epoch [25/25], step [1370/1875], loss-----> 0.043
Epoch [25/25], step [1371/1875], loss-----> 0.100
Epoch [25/25], step [1372/1875], loss-----> 0.042
Epoch [25/25], step [1373/1875], loss-----> 0.214


Epoch [25/25], step [1524/1875], loss-----> 0.333
Epoch [25/25], step [1525/1875], loss-----> 0.065
Epoch [25/25], step [1526/1875], loss-----> 0.099
Epoch [25/25], step [1527/1875], loss-----> 0.066
Epoch [25/25], step [1528/1875], loss-----> 0.030
Epoch [25/25], step [1529/1875], loss-----> 0.127
Epoch [25/25], step [1530/1875], loss-----> 0.029
Epoch [25/25], step [1531/1875], loss-----> 0.030
Epoch [25/25], step [1532/1875], loss-----> 0.044
Epoch [25/25], step [1533/1875], loss-----> 0.040
Epoch [25/25], step [1534/1875], loss-----> 0.055
Epoch [25/25], step [1535/1875], loss-----> 0.014
Epoch [25/25], step [1536/1875], loss-----> 0.129
Epoch [25/25], step [1537/1875], loss-----> 0.117
Epoch [25/25], step [1538/1875], loss-----> 0.033
Epoch [25/25], step [1539/1875], loss-----> 0.071
Epoch [25/25], step [1540/1875], loss-----> 0.044
Epoch [25/25], step [1541/1875], loss-----> 0.049
Epoch [25/25], step [1542/1875], loss-----> 0.047
Epoch [25/25], step [1543/1875], loss-----> 0.022


Epoch [25/25], step [1691/1875], loss-----> 0.102
Epoch [25/25], step [1692/1875], loss-----> 0.068
Epoch [25/25], step [1693/1875], loss-----> 0.092
Epoch [25/25], step [1694/1875], loss-----> 0.028
Epoch [25/25], step [1695/1875], loss-----> 0.053
Epoch [25/25], step [1696/1875], loss-----> 0.126
Epoch [25/25], step [1697/1875], loss-----> 0.129
Epoch [25/25], step [1698/1875], loss-----> 0.108
Epoch [25/25], step [1699/1875], loss-----> 0.013
Epoch [25/25], step [1700/1875], loss-----> 0.185
Epoch [25/25], step [1701/1875], loss-----> 0.054
Epoch [25/25], step [1702/1875], loss-----> 0.041
Epoch [25/25], step [1703/1875], loss-----> 0.039
Epoch [25/25], step [1704/1875], loss-----> 0.030
Epoch [25/25], step [1705/1875], loss-----> 0.034
Epoch [25/25], step [1706/1875], loss-----> 0.067
Epoch [25/25], step [1707/1875], loss-----> 0.115
Epoch [25/25], step [1708/1875], loss-----> 0.075
Epoch [25/25], step [1709/1875], loss-----> 0.051
Epoch [25/25], step [1710/1875], loss-----> 0.067


Epoch [25/25], step [1861/1875], loss-----> 0.170
Epoch [25/25], step [1862/1875], loss-----> 0.094
Epoch [25/25], step [1863/1875], loss-----> 0.072
Epoch [25/25], step [1864/1875], loss-----> 0.076
Epoch [25/25], step [1865/1875], loss-----> 0.182
Epoch [25/25], step [1866/1875], loss-----> 0.289
Epoch [25/25], step [1867/1875], loss-----> 0.142
Epoch [25/25], step [1868/1875], loss-----> 0.154
Epoch [25/25], step [1869/1875], loss-----> 0.187
Epoch [25/25], step [1870/1875], loss-----> 0.052
Epoch [25/25], step [1871/1875], loss-----> 0.087
Epoch [25/25], step [1872/1875], loss-----> 0.079
Epoch [25/25], step [1873/1875], loss-----> 0.075
Epoch [25/25], step [1874/1875], loss-----> 0.067


In [12]:
with torch.no_grad():
    right = 0
    total = 0
    for images, labels in test_loader:
        images  = images.reshape(-1,784).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, pred = torch.max(outputs.data, 1)
        total+= labels.size(0)
        right+=(pred==labels).sum().item()
    print('Test Accuracy  .....> {:.3f}'.format(100*right/total))

    torch.save(model.state_dict(), '../datasets/models/torch103.ckpt')
        

Test Accuracy  .....> 97.020
